<a href="https://colab.research.google.com/github/jaegon-kim/python_study/blob/main/src/ai_essential_250317/house_price_prediction/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Price Prediction
- **목표**
  - 이 워크샵은 주어진 데이터셋을 이용해 심층신경망 모델을 학습시켜 주택의 최종 판매 가격(SalePrice)을 예측하는 것이 최종 목표입니다.

- **데이터셋 정보**
  - 데이터셋은 총 79개의 설명 변수와 타겟 변수인 주택 가격(SalePrice)로 구성됩니다.
  - 설명 변수는 주택의 다양한 특성(예: 건축 연도, 면적, 위치, 방 개수 등)을 포함합니다.
  - 데이터는 판매 가격이 포함된 학습용 데이터인 `X`, `y` 와 판매 가격이 포함되지 않은 평가용 데이터인 `TEST`파일로 나뉘며, 각각 모델 학습 및 평가에 사용됩니다.
    - 평가용 데이터 `TEST`의 판매 가격(SalePrice)를 예측 후 리더보드로 제출하여 평가합니다.

- **문제 유형**
  - 이 워크샵은 회귀 문제로 연속형 변수를 예측하는 것이 목표입니다. 모델의 성능은 `Mean Absolute Error`로 측정됩니다.

## 1. 환경 설정

In [ ]:
%%capture
!pip install JAEN -U

In [ ]:
# 그대로 실행하세요.
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = "김재곤"
assert username, "username 변수에 값이 설정되지 않았습니다."

In [ ]:
# 그대로 실행하세요.
from JAEN.competition import Competition
comp = Competition(
    username=username,
    course_name='AI Essential',
    course_round='0317(1)',
    competition_name='House Price Prediction'
)

## 2. 데이터 로드

In [ ]:
from JAEN.datasets import load_house_price
X, y, TEST = load_house_price()
X.shape, y.shape, TEST.shape

(torch.Size([1460, 79]), torch.Size([1460, 1]), torch.Size([1459, 79]))

## 3. 제출 예시 코드

In [ ]:
# TEST의 예측값 대입 (지금은 0으로 채워진 값 대입)
comp.prediction =  torch.zeros(1459)
comp.prediction

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
# 제출
comp.submit()

[House Price Prediction 평가 결과]
 AI Essential 0317(1) 과정 김재곤님의 점수는 180493.328125 입니다.

## 4. 심층신경망 모델을 구성하고 학습하여 TEST를 예측해보세요.
- TEST의 예측 결과는 `comp.prediction`에 대입해주세요. **torch.tensor** 형태여야합니다.

In [ ]:
# DataLoader 생성
from torch.utils.data import DataLoader, TensorDataset
train_loader = DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True)

In [ ]:
# DNN 모델 구성
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(79, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = DNN().to(device)
summary(model)


Layer (type:depth-idx)                   Param #
DNN                                      --
├─Linear: 1-1                            10,240
├─Linear: 1-2                            8,256
├─Linear: 1-3                            65
├─ReLU: 1-4                              --
Total params: 18,561
Trainable params: 18,561
Non-trainable params: 0

In [ ]:
# 손실함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
X.shape

torch.Size([1460, 79])

In [ ]:
# 모델 학습 과정 구현
epochs = 1000
for epoch in range(epochs):
    running_loss = 0.0
    model.train()

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}')

Epoch [1/1000], Loss: 39080812944.695656
Epoch [101/1000], Loss: 1787066185.7391305
Epoch [201/1000], Loss: 1054865424.0
Epoch [301/1000], Loss: 782635552.6956521
Epoch [401/1000], Loss: 641086149.9130435
Epoch [501/1000], Loss: 539276845.5652174
Epoch [601/1000], Loss: 464331098.0869565
Epoch [701/1000], Loss: 403049870.26086956
Epoch [801/1000], Loss: 358245848.0
Epoch [901/1000], Loss: 321396879.2173913


In [ ]:
# 학습된 모델의 TEST 예측
model.eval()
with torch.no_grad():
    outputs = model(TEST)
outputs

tensor([[129223.4766],
        [208822.5938],
        [181819.0938],
        ...,
        [182610.2031],
        [119387.1484],
        [265305.9375]])

In [ ]:
# comp.prediction에 TEST 예측 결과 대입
comp.prediction = outputs
comp.prediction

tensor([[129223.4766],
        [208822.5938],
        [181819.0938],
        ...,
        [182610.2031],
        [119387.1484],
        [265305.9375]])

In [ ]:
# 제출
comp.submit()

[House Price Prediction 평가 결과]
 AI Essential 0317(1) 과정 김재곤님의 점수는 18225.046875 입니다.